In [0]:
import re
import matplotlib.pyplot as plot
import pandas as pd
from keras import Sequential
from keras.constraints import maxnorm
from keras.layers import Embedding, Conv1D, Dropout, MaxPooling1D, Flatten, Dense, LSTM, GlobalMaxPooling1D
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [0]:
train_movie_df = pd.read_csv('train.tsv', delimiter='\t', encoding='utf-8')
test_movie_df = pd.read_csv('test.tsv', delimiter='\t', encoding='utf-8')

In [0]:
train_movie_df = train_movie_df.drop(columns=['PhraseId', 'SentenceId'])
label=train_movie_df[['Sentiment']]
train_movie_df=train_movie_df.drop(columns=['Sentiment'])
test_movie_df = test_movie_df.drop(columns=['PhraseId', 'SentenceId'])

In [0]:
train_movie_df['Phrase'] = train_movie_df['Phrase'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x.lower()))
test_movie_df['Phrase'] = test_movie_df['Phrase'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x.lower()))

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train_movie_df['Phrase'].values)
X_train = tokenizer.texts_to_sequences(train_movie_df['Phrase'].values)
X_train = pad_sequences(X_train)

In [25]:
X_train.shape

(156060, 45)

In [0]:
max_fatures = 1000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(test_movie_df['Phrase'].values)
X_test = tokenizer.texts_to_sequences(test_movie_df['Phrase'].values)
X_test = pad_sequences(X_test)

In [27]:
X_test.shape

(66292, 45)

In [0]:
embed_dim = 128
# lstm_out = 196 
num_classes = 5

In [0]:
def create_model():
    model = Sequential()
    model.add(Embedding(13734, embed_dim, input_length=X_train.shape[1]))
    model.add(Dropout(0.2))
    model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [37]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label)
Y_train = to_categorical(integer_encoded)
X_tr, X_te, Y_tr, Y_te = train_test_split(X_train, Y_train, test_size=0.25, random_state=30)
print(X_tr.shape,Y_tr.shape)
print(X_te.shape,Y_te.shape)

(117045, 45) (117045, 5)
(39015, 45) (39015, 5)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
model = create_model()

In [39]:
history = model.fit(X_tr, Y_tr, epochs=5, batch_size=512, validation_data=(X_te,Y_te))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 117045 samples, validate on 39015 samples
Epoch 1/5
117045/117045 [==============================] - 36s 308us/step - loss: 1.1599 - accuracy: 0.5512 - val_loss: 1.0128 - val_accuracy: 0.6033
Epoch 2/5
117045/117045 [==============================] - 36s 306us/step - loss: 0.9832 - accuracy: 0.6157 - val_loss: 0.9772 - val_accuracy: 0.6161
Epoch 3/5
117045/117045 [==============================] - 36s 305us/step - loss: 0.9318 - accuracy: 0.6354 - val_loss: 0.9389 - val_accuracy: 0.6261
Epoch 4/5
117045/117045 [==============================] - 36s 304us/step - loss: 0.8920 - accuracy: 0.6512 - val_loss: 0.9244 - val_accuracy: 0.6321
Epoch 5/5
117045/117045 [==============================] - 36s 305us/step - loss: 0.8612 - accuracy: 0.6629 - val_loss: 0.9183 - val_accuracy: 0.6349


VALIDATION ON TEST DATA SET

In [40]:
y_pred=model.predict_classes(X_test[:1])
print(y_pred[0]," PREDICTED LABEL")

2  PREDICTED LABEL


In [41]:
sub_file = pd.read_csv('sampleSubmission.csv',sep=',')
print(sub_file['Sentiment'].iloc[0]," ACTUAL LABEL")

2  ACTUAL LABEL


QUESTION 6: TUNE THE HYPER PARAMETERS

In [0]:
from keras.optimizers import adam
s=adam(lr=0.001)
model1= Sequential()
model1.add(Embedding(13734,50,input_length=X_train.shape[1]))
model1.add(Dropout(0.2))
model1.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128,activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='binary_crossentropy',optimizer=s,metrics=['accuracy'])

In [45]:
history1 = model1.fit(X_tr, Y_tr, epochs=5, batch_size=512, validation_data=(X_te,Y_te))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 117045 samples, validate on 39015 samples
Epoch 1/5
117045/117045 [==============================] - 19s 161us/step - loss: 0.3959 - accuracy: 0.8316 - val_loss: 0.3748 - val_accuracy: 0.8426
Epoch 2/5
117045/117045 [==============================] - 19s 160us/step - loss: 0.3531 - accuracy: 0.8475 - val_loss: 0.3415 - val_accuracy: 0.8496
Epoch 3/5
117045/117045 [==============================] - 19s 160us/step - loss: 0.3327 - accuracy: 0.8537 - val_loss: 0.3362 - val_accuracy: 0.8498
Epoch 4/5
117045/117045 [==============================] - 19s 161us/step - loss: 0.3241 - accuracy: 0.8573 - val_loss: 0.3298 - val_accuracy: 0.8531
Epoch 5/5
117045/117045 [==============================] - 19s 162us/step - loss: 0.3171 - accuracy: 0.8605 - val_loss: 0.3257 - val_accuracy: 0.8552
